<a href="https://colab.research.google.com/github/sebas0907/Python-projects/blob/master/Kopie_von_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

In [ ]:
#function to load and split data
def load_data(path):
    texts, labels = [], []
    with open(path) as f:
        for line in f:
            split = line.split()
            labels.append(split[0].strip())
            texts.append(' '.join(split[1:]).strip())
    return texts, labels

In [ ]:
#define train and test data with labels
train_set, train_label = load_data(train_file_path)
test_set, test_label = load_data(test_file_path)


In [ ]:
#tokenizer
tokenizer = keras.preprocessing.text.Tokenizer()
def token(text):
  sequences =  tokenizer.fit_on_texts(text)
  return sequences

In [ ]:
training = token(train_set)
testing = token(test_set)

In [ ]:
#text to sequences
train = tokenizer.texts_to_sequences(train_set)
test = tokenizer.texts_to_sequences(test_set)

In [ ]:
#pad sequences
train = np.array(train)
test = np.array(test)
train = keras.preprocessing.sequence.pad_sequences(train)
test = keras.preprocessing.sequence.pad_sequences(test)

In [ ]:
#size of the vocabulary
vocab_size = len(tokenizer.word_index)

In [ ]:
#map labels to integers
label_int = {'ham':0, 'spam':1}

In [ ]:
#map integers to labels
int_label = {0:'ham', 1:'spam'}

In [ ]:
#one-hot enconding function
def one_hot(tags):
  label = np.array(tags)
  one_hot_label = np.zeros(len(label))
  for i in range(len(label)):
    one_hot_label[i] = label_int[label[i]]
  categories = keras.utils.to_categorical(one_hot_label)
  return categories

In [ ]:
#encoded labels
labels = one_hot(train_label)
t_labels = one_hot(test_label)

In [ ]:
#create the model
model = tf.keras.Sequential()
model.add(keras.layers.Embedding(vocab_size+1, 189))
model.add(keras.layers.LSTM(89, recurrent_dropout=0.2)) #
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(train, labels, validation_data=(test, t_labels), shuffle=True, epochs=10, steps_per_epoch=15)

In [ ]:
results = model.evaluate(test, t_labels)

In [ ]:
''# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  sequences = tokenizer.texts_to_sequences([pred_text])
  sequences = keras.preprocessing.sequence.pad_sequences(sequences)
  predicted = model.predict(sequences)[0]
  predicted = [np.max(predicted), int_label[np.argmax(predicted)]]

  return (predicted)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
